In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,math,itertools,datetime,time,logging,os

import torch
import pandas as pd

from pytorch_pretrained_bert import BertTokenizer,BertModel,BertForMaskedLM

from bs4 import BeautifulSoup
#logging.basicConfig(level=logging.INFO)

In [2]:
basedir = '..\\..\\TF_data\\BLOGENTRIES\\blogs\\'

files = os.listdir(basedir)[:250]

df = pd.DataFrame()
for c_file in files:
    
    try:
        with open(basedir+c_file) as fp: soup = BeautifulSoup(fp, 'xml')
    except:
        continue
    cells = soup.findAll(['date','post'])

    data = []
    for k in range(0,len(cells),2):
        data.append([
            str(cells[k]).replace('<date>','').replace('</date>','').replace(',','-'),
            str(cells[k+1]).replace('<post>','').replace('</post>','').replace('\n','').replace('\t','').lstrip().rstrip()
        ])
    cf = pd.DataFrame(data).rename(columns={0:'Date Post',1:'Text Post'})
    c_ID,c_Gender,c_Age,c_Topic,c_AstroSign = c_file.split('.')[:-1]

    cf['ID Poster'] = c_ID
    cf['Gender Poster'] = c_Gender
    cf['Age Poster'] = c_Age
    cf['Topic Channel'] = c_Topic
    cf['Astro Sign Poster'] = c_AstroSign
    
    df = pd.concat([df,cf]).reset_index(drop=True)

In [3]:
df

,Date Post,Text Post,ID Poster,Gender Poster,Age Poster,Topic Channel,Astro Sign Poster
0,31-May-2004,"Well, everyone got up and going this morning. ...",1000331,female,37,indUnk,Leo
1,29-May-2004,My four-year old never stops talking. She'll ...,1000331,female,37,indUnk,Leo
2,28-May-2004,"Actually it's not raining yet, but I bought 15...",1000331,female,37,indUnk,Leo
3,28-May-2004,Ha! Just set up my RSS feed - that is so easy!...,1000331,female,37,indUnk,Leo
4,28-May-2004,"Oh, which just reminded me, we were talking ab...",1000331,female,37,indUnk,Leo
...,...,...,...,...,...,...,...
30086,01-January-2004,Happy new year to one and all. It appears the...,1222749,male,37,indUnk,Capricorn
30087,13-February-2004,Sad News urlLink New Zealand Backpacker kil...,1222749,male,37,indUnk,Capricorn
30088,01-February-2004,"Forget About Tomorrow Calling, distorting...",1222749,male,37,indUnk,Capricorn
30089,25-March-2004,""" Into the below"" "" Ty Ray OK "" ...",1222749,male,37,indUnk,Capricorn


In [61]:
df.loc[2,'Text Post']

"Actually it's not raining yet, but I bought 15 tickets to the  urlLink Goldeyes  game for my Mom's birthday tonight, and it is supposed to rain.  Do they cancel baseball games because of rain?  Although the ballpark is beautiful, it ain't the  urlLink SkyDome .  We used to go to the Jays games occassionally when we lived in Toronto and really like taking the kids to the Goldeyes games now.  I don't know what  urlLink Blue Jays  tickets cost now, but I'm sure it's cheaper here in Winnipeg.  Oh, I just checked and it  definitely  is!"

In [129]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

#model.eval()

In [130]:
#text = '[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]'

#'[CLS] Actually its not raining yet but I bought 15 tickets to the game for my Moms birthday tonight, and it is supposed to rain. [SEP] Do they cancel baseball games because of rain? [SEP]'

text = '[CLS] Actually its not raining yet but I bought 15 tickets to the game for my Moms birthday tonight, and it is supposed to rain. [SEP] Do they cancel baseball games because of rain [MASK]'
tok_text = tokenizer.tokenize(text)

In [131]:
mask_ind = [k for k in range(len(tok_text)) if tok_text[k]=='[MASK]']

In [132]:
ind_tokens = tokenizer.convert_tokens_to_ids(tok_text)

In [133]:
tokens_tensor = torch.tensor([ind_tokens])

In [134]:
with torch.no_grad():
    outputs = model(tokens_tensor)
    pred = outputs[0]

In [135]:
for c_ind in mask_ind:
    c_token_ind = torch.argsort(pred[c_ind],descending=True)[:10].tolist()
    print(tokenizer.convert_ids_to_tokens(c_token_ind))

['?', ',', 'or', '.', 'and', 'but', 'because', 'like', 'then', 'not']


In [128]:
text

'[CLS] Actually its not raining yet but I bought 15 tickets to the game for my Moms birthday tonight, and it is supposed to rain. [SEP] Do they cancel baseball games because of rain [MASK] [MASK] [MASK] [MASK]'